In [1]:
# !pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from google.colab import drive, files
drive.mount('/content/drive')
rootpath = '/content/drive/Shareddrives/KW phase1 repo'

Mounted at /content/drive


In [3]:
import time
def current_milli_time():
    return round(time.time() * 1000)

In [4]:
import sys
sys.path.append(rootpath+'/package')
from utils.env_sdp import env
import pickle
import numpy as np
import csv

In [5]:
env.N

16

In [ ]:
from model.uav import UAV, setUAVEnv
from model.sdp import calculateSDP, setSDPEnv
from model.pathlossmodel import PathLossModel, NoiseMaker, setPathLossModelEnv

from service.distributeuavs import createUAVsSamples,setDistributeUavsEnv , TestEnv
from service.measurerss import measureRSS, setMeasuerRssEnv
from service.combinationset import combinationSet , setCombinationSetEnv
from multiprocessing import Pool

# setDistributeUavsEnv("sdp")
# TestEnv()
# setCombinationSetEnv("sdp")
# setPathLossModelEnv("sdp")
# setMeasuerRssEnv("sdp")
# setUAVEnv("sdp")
# setSDPEnv("sdp")

In [ ]:
savePath = '/content/drive/Shareddrives/KW phase1 repo/datas/anomalyNewSDPResult/0817_PCA'
rootPath = '/content/drive/Shareddrives/KW phase1 repo'
!mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/anomalySDPResultPCA
!mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/anomalySDPResultPCA/0817_PCA

mkdir: cannot create directory ‘/content/drive/Shareddrives/KW phase1 repo/datas/anomalySDPResultPCA’: File exists
mkdir: cannot create directory ‘/content/drive/Shareddrives/KW phase1 repo/datas/anomalySDPResultPCA/0817_PCA’: File exists


In [ ]:
# savePath = '/content/drive/Shareddrives/KW phase1 repo/datas/anomalySDPResultPCA/done0817'
# rootPath = '/content/drive/Shareddrives/KW phase1 repo'
# !mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/anomalySDPResultPCA
# !mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/anomalySDPResultPCA/done0817

In [ ]:
deviations = [2]

In [ ]:
GetDevicationStr = lambda deviation: str(int(deviation*100))

In [ ]:
# pathLossModel = PathLossModel()
# pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = env.noiseDeviation)
# !mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/uavPosition/300_0706
# !mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/uavPosition/300_0706/deviation
# for deviation in deviations:
#   deviationStr =  GetDevicationStr(deviation)
#   !mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/uavPosition/300_0706/deviation/$deviationStr
#   createUAVsSamples(pathLossModel, rootPath+"/datas/uavPosition/300_0706/deviation/"+deviationStr)
#   print()

In [ ]:
import pickle as dill
from model.uav import UAV
MAXPICKERROR = 3
for deviation in deviations:
  for ipickError in np.arange(0,MAXPICKERROR+1,1):
    import pickle as dill
    file = open(savePath+"/anomalyUAVs_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_100.npy","wb")
    file.close()
    file = open(savePath+"/UAVsSamples_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_100.npy","wb")
    file.close()
    for j in range(env.N_ngps):
      file = open(savePath+"/test_position{}_pickerror{}_deviation{}_100.csv".format(j,ipickError,GetDevicationStr(deviation)),"wb")
      file.close()
      print(ipickError)
for deviation in deviations:
  for ipickError in np.arange(0,MAXPICKERROR+1,1):
    _pathLossModel = PathLossModel()
    _pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = deviation) 
    with open(rootPath+"/datas/uavPosition/300_0706/done_deviation/"+GetDevicationStr(deviation)+"/UAVsSamples.pkl", "rb") as f:
      UAVsSamples = pickle.load(f)
    # print(len(UAVsSamples[0]))
    # UAVsSamples = {0:UAVsSamples[0]}
    for iSample, UAVSample in UAVsSamples.items():
      for pickTime in range(ipickError):
        print("pickTime",pickTime)
        uavList_gps = []
        for id, uav in UAVSample.items():
          if not uav.observedPosition == None and uav.observedPosition == uav.realPosition:
            uavList_gps.append(id)
        pickedAnomalyIndex = np.random.choice(uavList_gps, 1, replace=False)[0]
        # print("pickedAnomalyIndex",pickedAnomalyIndex)
        originObservedPosition = UAVSample[pickedAnomalyIndex].observedPosition
        shift = np.random.choice(np.arange(500, 1000), 3, replace=False)
        anomalyObservedPosition = [originObservedPosition[0]+shift[0], originObservedPosition[1]+shift[1], originObservedPosition[2]+shift[2]]
        UAVSample[pickedAnomalyIndex].observedPosition = anomalyObservedPosition
    measureRSS(UAVsSamples, _pathLossModel)
    for iSample, UAVSample in UAVsSamples.items():
      uavAnchorOfSegments = []
      uavTargetOfSegments = []
      uavAnchorOfSegmentsPCA = []
      uavTargetOfSegmentsPCA = []
      groupSet = combinationSet({0:UAVSample})[0]
      # print(groupSet)

      ## for GPS data
      for anchorSize, segments in groupSet.items():
        for segment in segments:
          uavAnchors = []
          for anchorID in segment:
            uavAnchors.append(UAVSample[anchorID].observedPosition)
          for i in range(env.N_gps-len(uavAnchors)):
            uavAnchors.append([0.0,0.0,0.0])
          uavAnchorOfSegments.append(uavAnchors)
      # print(np.array(uavAnchorOfSegments).shape)
      ## for RSS data
      for anchorSize, segments in groupSet.items():
        for segment in segments:
          uavTargets = [ [] for i in range(env.N_gps)]
          iAnchor = 0
          for anchorID in segment:
            for id, uav in UAVSample.items():
              if uav.observedPosition == None:
                rss = uav.searchRSSById(anchorID)
                uavTargets[iAnchor].append(rss)
            iAnchor += 1
          for i in range(env.N_gps-iAnchor):
            # print(iAnchor)
            for id, uav in UAVSample.items():
              if uav.observedPosition == None:
                rss = uav.searchRSSById(anchorID)
                uavTargets[iAnchor].append(0)
            iAnchor += 1
          # print(uavTargets)
          # print(np.array(uavTargets).shape)
          uavTargetOfSegments.append(uavTargets)
      # print(np.array(uavTargetOfSegments).shape)
      import numpy as np
      from sklearn.decomposition import PCA
      for anchorGPS in uavAnchorOfSegments:
        pca = PCA(n_components=1)
        anchorGPStransfer = pca.fit(anchorGPS).transform(anchorGPS)
        uavAnchorOfSegmentsPCA.append(anchorGPStransfer)
      # print(np.array(uavAnchorOfSegmentsPCA).shape)
      for anchorRSS in uavTargetOfSegments:
        pca = PCA(n_components=1)
        anchorRSStransfer = pca.fit(anchorRSS).transform(anchorRSS)
        uavTargetOfSegmentsPCA.append(anchorRSStransfer)
      # print(np.array(uavTargetOfSegmentsPCA).shape)
      uavAnchorOfSegmentsPCA = np.array(uavAnchorOfSegmentsPCA)
      uavTargetOfSegmentsPCA = np.array(uavTargetOfSegmentsPCA)
      PCAdata = np.append(uavAnchorOfSegmentsPCA, uavAnchorOfSegmentsPCA,axis=2)
      PCAdata = PCAdata.transpose((1,0,2))
      # print(np.array(PCAdata).shape)
      for j in range(env.N_ngps):
        with open(savePath+"/test_position{}_pickerror{}_deviation{}_100.csv".format(j,ipickError,GetDevicationStr(deviation)), "a") as _csv:
          csvWriter = csv.writer(_csv, delimiter=',')
          csvWriter.writerows(PCAdata[j])
      anomalyUAVs = []
      for iUAV, UAV in UAVSample.items():
        if UAV.observedPosition != None and UAV.observedPosition != UAV.realPosition:
          # print(UAV.observedPosition)
          # print(UAV.realPosition)
          anomalyUAVs.append(UAV.id)
      with open(savePath+"/anomalyUAVs_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_100.npy", 'ab') as handle:
        dill.dump(anomalyUAVs, handle)
      with open(savePath+"/UAVsSamples_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_100.npy", 'ab') as handle:
        dill.dump(UAVSample, handle)
  

0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
pickTime 0
p